In [ ]:
# Mapper
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import sqlalchemy as sa
#from ingester3.config import source_db_path
#from ingester3.Country import Country
#from ingester3.extensions import *
#from ingester3.ViewsMonth import ViewsMonth

from views_dataviz.map import mapper, utils
from views_dataviz import color
from views_dataviz.map.presets import ViewsMap
from views_dataviz.map import utils
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


#To make an API call 
#Importing the packages

import requests
import json
import os

home = os.path.expanduser("~")

# Specify where you want to save the data, else it is saved to your desktop. 
#path = home+'/Desktop/'

path = home+'/Desktop/' 

#This url is the base 
api_location = 'https://api.viewsforecasting.org/'

localgitpath = '/Users/chandlerwilliams/Desktop/VIEWS_new/FCDO_predicting_fatalities/'

In [ ]:
run = 'fatalities001_2022_11_t01'
loa = 'pgm'
tv = 'sb' # or 'osv' or 'ns'
model = 'sc_pgm_sb_main'


url = api_location+ run+ '/'+ loa+ '/'+ tv+'/'+ model

print(url)

# Request function
r = requests.get(url)
PAGE_SIZE=1000
master_list=[]
r=requests.get(url)
page_data=r.json()
master_list+=page_data['data']
    
while page_data['next_page'] != '':
    r=requests.get(page_data['next_page'])
    page_data=r.json()
    master_list+=page_data['data']
        
df=pd.DataFrame(master_list)

df.to_csv(path+'latest_run.csv', sep=';', encoding='utf-8', index=True)

display(df)

In [ ]:
df1 = df.rename(columns = {'pg_id':'priogrid_gid'})

In [ ]:
# Common parameters:

dev_id = 'fatalities001'
run_id = 'fatalities001' 
EndOfHistory = 515
prod_id = '2022_11_t01'
get_future = True
WriteToOverleaf = False

level = 'pgm'

In [ ]:
class Mapper2:
    """
    `Map` takes basic properties and allows the user to consecutively add
    layers to the Map object. This makes it possible to prepare mapping
    "presets" at any level of layeredness that can be built on further.
    
    Mapper2 allows for the customizable addition of scaling to the map. 
    -re-add the code for labels later when i can test it

    Attributes
    ----------
    width: Integer value for width in inches.
    height: Integer value for height in inches.
    bbox: List for the bbox per [xmin, xmax, ymin, ymax].
    frame_on: Bool for whether to draw a frame around the map.
    title: Optional default title at matplotlib's default size.
    figure: Optional tuple of (fig, size) to use if you want to plot into an
        already existing fig and ax, rather than making a new one.
    """

    def __init__(
        self,
        width,
        height,
        bbox=None,
        cmap=None,
        frame_on=True,
        title="",  # Default title without customization. (?)
        figure=None,
    ):
        self.width = width
        self.height = height
        self.bbox = bbox  # xmin, xmax, ymin, ymax
        self.cmap = cmap
        if figure is None:
            self.fig, self.ax = plt.subplots(figsize=(self.width, self.height))
        else:
            self.fig, self.ax = figure
        self.texts = []
        self.ax.set_title(title)

        if frame_on:  # Remove axis ticks only.
            self.ax.tick_params(
                top=False,
                bottom=False,
                left=False,
                right=False,
                labelleft=False,
                labelbottom=False,
            )
        else:
            self.ax.axis("off")

        if bbox is not None:
            self.ax.set_xlim((self.bbox[0], self.bbox[1]))
            self.ax.set_ylim((self.bbox[2], self.bbox[3]))

    def add_layer(self, gdf, map_scale=False, map_dictionary=False, cmap=None, inform_colorbar=False, **kwargs):
        """Add a geopandas plot to a new layer.

        Parameters
        ----------
        gdf: Geopandas GeoDataFrame to plot.
        cmap: Optional matplotlib colormap object or string reference
            (e.g. "viridis").
        inform_colorbar: Set or overwrite colorbar with the current layer.
            Not applicable when `color` is supplied in the kwargs.
        map_scale: set a manual scale for the map. If missing defaults to the Remco procedure. 
        map_dictionary: set manual labels for the map. If missing defaults to the default labels.
        **kwargs: Geopandas `.plot` keyword arguments.
        """
        if "color" in kwargs:
            colormap = None
        else:
            colormap = self.cmap if cmap is None else cmap
            if inform_colorbar and "column" in kwargs:
                if hasattr(self, "cax"):
                    self.cax.remove()
                if "vmin" not in kwargs:
                    self.vmin = gdf[kwargs["column"]].min()
                else:
                    self.vmin = kwargs["vmin"]
                if "vmax" not in kwargs:
                    self.vmax = gdf[kwargs["column"]].max()
                else:
                    self.vmax = kwargs["vmax"]
        
        try: Mapper2.add_colorbar(self, colormap, min(map_scale), max(map_scale))
        except: Mapper2.add_colorbar(self, colormap, self.vmin, self.vmax)
        
        try:
            self.ax = gdf.plot(ax=self.ax, cmap=colormap, vmin=min(map_scale), vmax=max(map_scale), **kwargs)
        except: 
            self.ax = gdf.plot(ax=self.ax, cmap=colormap, **kwargs)

                
        return self
    
    def add_colorbar(
        self,
        cmap,
        vmin,
        vmax,
        location="right",
        size="5%",
        pad=0.1,
        alpha=1,
        labelsize=16,
        tickparams=None,
    ):
        """Add custom colorbar to Map.

        Needed since GeoPandas legend and plot axes do not align, see:
        https://geopandas.readthedocs.io/en/latest/docs/user_guide/mapping.html

        Parameters
        ----------
        cmap: Matplotlib colormap object or string reference (e.g. "viridis").
        vmin: Minimum value of range colorbar.
        vmax: Maximum value of range colorbar.
        location: String for location of colorbar: "top", "bottom", "left"
            or "right".
        size: Size in either string percentage or number of pixels.
        pad: Float for padding between the plot's frame and colorbar.
        alpha: Float for alpha to apply to colorbar.
        labelsize: Integer value for the text size of the ticklabels.
        tickparams: Dictionary containing value-label pairs. For example:
            {0.05: "5%", 0.1: "10%"}
        """
        norm = plt.Normalize(vmin, vmax)
        if isinstance(cmap, str):
            cmap = plt.get_cmap(cmap)
        cmap = color.force_alpha_colormap(cmap=cmap, alpha=alpha)
        scalar_to_rgba = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        divider = make_axes_locatable(self.ax)
        self.cax = divider.append_axes(location, size, pad)
        self.cax.tick_params(labelsize=labelsize)
        tickvalues = (
            list(tickparams.keys()) if tickparams is not None else None
        )
        self.cbar = plt.colorbar(
            scalar_to_rgba, cax=self.cax, ticks=tickvalues
        )
        if tickparams is not None:
            self.cbar.set_ticklabels(list(tickparams.values()))
        return self
    
    def save(
        self, path, dpi=200, **kwargs
    ):  # Just some defaults to reduce work.
        """Save Map figure to file.
        Parameters
        ----------
        path: String path, e.g. "./example.png".
        dpi: Integer dots per inch. Increase for higher resolution figures.
        **kwargs: Matplotlib `savefig` keyword arguments.
        """
        self.fig.savefig(path, dpi=dpi, bbox_inches="tight", **kwargs)
        plt.close(self.fig)
        

def vid2date(i):
    year=str(1980 + i//12)
    month=str(i%12)
    return year+'/'+month
        
#def vid2date(i):
#    year=str(ViewsMonth(i).year)
#    month=str(ViewsMonth(i).month)
#    return year+'/'+month

#note the zip function occured earlier
standard_scale = [np.log1p(0),np.log1p(3),np.log1p(10), np.log1p(30), np.log1p(100),  np.log1p(300)]#, np.log1p(1000), np.log1p(3000),  np.log1p(10000)]
standard_scale_labels = ['0', '3','10', '30','100', '300']#, '1000', '3000', '10000']

small_scale=[np.log1p(0),np.log1p(3),np.log1p(10), np.log1p(30), np.log1p(100),  np.log1p(300)]#, np.log1p(1000)]


small_scale_labels = ['0', '3','10', '30','100', '300']#, '1000']

small_scale_nolabels = ['', '','', '','', '', '']

In [ ]:
# get pgm geometries
gdf_base = gpd.read_parquet(localgitpath+ 'geometry/pgm_geometry.parquet')

In [ ]:
# get cm geometries
gdf_c = gpd.read_parquet(localgitpath+ 'geometry/cm_geometry.parquet')
gdf_c = gdf_c.to_crs(4326)

In [ ]:
# Future prediction maps, predictions, rolling

stepstoplot=[3,5,6,8,12,18,24,36]

In [ ]:
df = df1
gdf2 = gdf_base.copy()

In [ ]:
df = df.set_index('priogrid_gid')

gdf2 = gdf2.set_index('priogrid_gid')

In [ ]:
joint = pd.merge(df, gdf2, left_index=True, right_index=True, how='left')
joint

In [ ]:
joint = joint.drop(['in_africa_x','in_me_x', 'geom_x'], axis=1)
joint = joint.rename(columns={'in_africa_y':'in_africa', 'in_me_y': 'in_me', 'geom_y':'geom'})
joint

In [ ]:
#df = df.join(gdf2.set_index("priogrid_gid"))#Seem to merge but the values dissapear from gdf2 

joint = gpd.GeoDataFrame(joint, geometry="geom")
joint = joint.reset_index()


In [ ]:
print(joint.loc[[64813]])

In [ ]:
for step in stepstoplot:
        month = step + EndOfHistory
        gdf = joint.loc[month]
        m=Mapper2(
        width=10,
        height=10,
        frame_on=True,
        title='Ensemble predictions as of ' + vid2date(EndOfHistory+step) + ', ' + str(step) + ' months after last month with data',
        bbox=[31.558968, 43.636326, -5.028795, 6.648430], 
        #bbox=[29.446846321370213, 42.987309710685814, -5.1561557161401845, 7.29970129951559], 
        ).add_layer(
        gdf=joint,
        map_scale=standard_scale,
        cmap="rainbow",
        edgecolor="black",
        linewidth=0.2,
        column='sc_pgm_sb_main', 
        inform_colorbar=True
        )
        ax=m.ax
        fg=gdf_c.plot(ax=ax,edgecolor='gray',linewidth=1.0,facecolor='None')
        figure=m.fig
        fontdict={'fontsize':20}
        figure.text(0.45,0.5,'KENYA',fontdict=fontdict,color='black')
        figure.text(0.65,0.8,'ETHIOPIA',fontdict=fontdict,color='black')
        figure.text(0.3,0.5,'UGANDA',fontdict=fontdict,color='black')
        figure.text(0.15,0.85,'S. SUDAN',fontdict=fontdict,color='black')
        figure.text(0.35,0.25,'TANZANIA',fontdict=fontdict,color='black')
        
        m.cbar.set_ticks(standard_scale)
        m.cbar.set_ticklabels(standard_scale_labels)

#        m.save(f'{overleafpath}Figures/Future/PredictionMap_cm_ensemble_standard_scale_r{EndOfHistory}_m{month}.png')

        m.save(f'{path}PredictionMap_Kenya_pgm_ensemble_standard_scale_r{EndOfHistory}_m{month}.png')